In [ ]:
!pip install langchain
!pip install getpass
!pip install langchain_openai
!pip install pypdf
!pip install chromadb
!pip install langchainhub
!pip install pypdf2
!pip install apify-client
!pip install flask-ngrok
!pip install pyngrok

In [ ]:
import getpass
import os
import bs4
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
import getpass
import os
import json
import threading
from flask import Flask, redirect, url_for, render_template, request, flash
from flask import Flask,jsonify
from werkzeug.utils import secure_filename
from pyngrok import ngrok, conf
conf.get_default().auth_token = "API_KEY"

def extract_and_merge_text(pdf_path):
    """
    Extracts text from all pages of a PDF and merges it into a single string.

    :param pdf_path: The path to the PDF file.
    :return: A single string containing the text of all pages.
    """
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"  # Adding a newline character for separation
    return text

# Define your desired data structure.
def model_predict(cvpdf_path):
  pdf_path = cvpdf_path
  mycv = extract_and_merge_text(pdf_path)
  json_format = """use only this format for the Answer as a json {\"Data_science_Rating":{}, \"Leadership_Rating":{}, \"Introduction":{}, ,\"Data_science_justifications":{},\"Leadership_justifications":{} , \"Recommendation1:{}, \"Recommendation2:{}, \"Recommendation3:{}, \"CourseRecommendation:{}, \"CourseProvider:{} }"""
  return json.loads(rag_chain.invoke(mycv+" rate the cv " + json_format))

LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="API_KEY"
LANGCHAIN_PROJECT="eduLLM"

os.environ["OPENAI_API_KEY"] = "API_KEY"


In [ ]:
# Load, chunk and index the contents of the blog.

loader = PyPDFLoader("thinkstats.pdf")
# loader2 = PyPDFLoader("data science for business.pdf")
loader3 = PyPDFLoader("Understanding management and leadership styles author Chartered Management Institute.pdf")
loader4 = PyPDFLoader("Leadership Management Principles, Models and Theories author Manoj Kumar Sharma.pdf")
loader5 = PyPDFLoader("Foundations of Data Science.pdf")
loader6 = PyPDFLoader("Becoming a better leader author Routledge & CRC Press.pdf")

from bs4 import BeautifulSoup as Soup
url = 'https://executive.mit.edu/course-finder?prefn1=courseTopics&prefv1=Business%20Analytics'
loader2 = RecursiveUrlLoader(
    url=url, max_depth=10, extractor=lambda x: Soup(x, "html.parser").text
)
url = "https://www.exed.hbs.edu/program-finder?topic=Leadership"
loader7 = RecursiveUrlLoader(
    url=url, max_depth=10, extractor=lambda x: Soup(x, "html.parser").text
)
url = 'https://www.coursera.org/browse/data-science'
loader8 = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
)
url = 'https://www.coursera.org/search?query=leadership'
loader9 = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
)
url = 'https://www.edx.org/executive-education'
loader10 = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()
docs.extend(loader2.load())
docs.extend(loader3.load())
docs.extend(loader4.load())
docs.extend(loader5.load())
docs.extend(loader6.load())
docs.extend(loader7.load())
docs.extend(loader8.load())
docs.extend(loader9.load())
docs.extend(loader10.load())

docs = filter_complex_metadata(docs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings() )

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [ ]:
template = """

Use the following pieces of context to answer the question at the end. Never ask a question just explain.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Analyze a data science and leadership CV using a Retrieval-Augmented Generation approach, comparing it against key literature in both fields. Provide:

Ratings & justifications: Leadership and data science expertise on a 0 to 100 scale. Justifications. BE STRICT WITH THE RATING!
Recommendations (No more than three bullet points): Specific, actionable advice for professional growth as a data science director, highlighting strengths and areas to improve based on literature benchmarks.
Recommend one course to improve the candidate career to be the best data science leader.

Focus on clarity and direct applicability in your response.

Important to make sure that you "quote from the material provided."
{context}
Question: {question}

Helpful Answer:
"""

custom_rag_prompt = PromptTemplate.from_template(template)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

input_to_prompt_template = {
    "context": retriever | format_docs, "question": RunnablePassthrough()
}
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
vectorstore.delete_collection()

In [ ]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
app = Flask(__name__)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url


# Define Flask routes
@app.route("/")
def index():
    return htmlcont


@app.route('/upload', methods=['POST'])
def upload_file():
  print("im in this")
  file = request.files['file']
  xx= model_predict(file.filename)
  xx["Overall_Rating"] = (xx["Data_science_Rating"] + xx['Leadership_Rating'])/2
  print(xx)
  response_data = xx

  return jsonify(response_data)

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

In [ ]:
htmlcont = """
<html lang="en"><head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Education Recommender</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link href="https://fonts.googleapis.com/css2?family=Nunito:wght@200;300;400;600;700;800;900&amp;display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.15.4/css/all.css">
    <style>
        body {
            font-family: 'Nunito', sans-serif;
            padding-top: 20px;
            padding-bottom: 20px;
        }

        .card-header {
            background-color: #f8f9fc;
        }

        .progress {
            height: 20px;
        }

        .progress-bar {
            line-height: 20px;
        }

        .navbar {
            background-color: #fff;
            box-shadow: 0 2px 4px rgba(0,0,0,.1);
        }

        .container-fluid {
            max-width: 1200px;
        }

        .shadow-lg {
            box-shadow: 0 1rem 3rem rgba(0,0,0,.175)!important;
        }

        .btn-primary {
            background-color: #4e73df;
            border-color: #4e73df;
        }

        .fw-bold {
            font-weight: 700;
        }

        .text-primary {
            color: #4e73df !important;
        }
    </style>
</head>

<body id="page-top">

    <div id="wrapper">
        <div class="container-fluid">
            <!-- Navbar -->
            <nav class="navbar navbar-expand navbar-light topbar mb-4 static-top">
                <h3 class="text-primary fw-bold m-0" style="
    padding-right: 49px;
    padding-left: 19px;
">Team Infinity</h3>
                <h1 class="text-primary fw-bold ml-3" style="
    padding-left: 123px;
">AI Education Recommender</h1>
            </nav>

            <!-- Content -->
            <div class="row">
                <div class="col-lg-4 mb-4">
                    <div class="card shadow mb-4">
                        <div class="card-header py-3 d-flex justify-content-between align-items-center">
                            <h6 class="m-0 font-weight-bold text-primary">Upload Your Resume</h6>
                            <a class="btn btn-primary btn-sm" role="button" onclick="document.getElementById('fileInput').click();">
                                <i class="fas fa-upload text-white"></i> Upload
                            </a>
                            <input type="file" id="fileInput" style="display: none;" accept=".pdf">
                        </div><div id="loadingIndicator" class="text-center" style="padding: 20px;display: none;">
    <div class="spinner-border text-primary" role="status">
        <span class="visually-hidden">Loading...</span>
    </div>
    <p>Loading...</p>
</div>
                        <div class="card-body">
                            <!-- Progress bars -->
                            <h4 class="small fw-bold">Data Science <span class="float-end" id="Data_science_Rating_P"></span></h4>
                            <div class="progress mb-4">
                                <div class="progress-bar bg-danger" role="progressbar" style="width: 1%;" aria-valuenow="20" aria-valuemin="0" aria-valuemax="100" id="Data_science_Rating"></div>
                            </div>
                            <h4 class="small fw-bold">Leadership <span class="float-end" id="Leadership_Rating_P"></span></h4>
                            <div class="progress mb-4">
                                <div class="progress-bar bg-warning" role="progressbar" style="width: 1%;" aria-valuenow="40" aria-valuemin="0" aria-valuemax="100" id="Leadership_Rating"></div>
                            </div>
                            <h4 class="small fw-bold">Overall Score <span class="float-end" id="Overall_Rating_P"></span></h4>
                            <div class="progress">
                                <div class="progress-bar bg-primary" role="progressbar" style="width: 1%;" aria-valuenow="60" aria-valuemin="0" aria-valuemax="100" id="Overall_Rating"></div>
                            </div>
                        </div>
                    </div>
                </div>

                <div class="col">
                    <div class="card mb-4 shadow-lg">
                        <div class="card-body">
                            <h4 class="card-title">Candidate Summary</h4>
                            <p class="card-text" id="Introduction"></p>
                            <p class="card-text" id="Data_science_justifications"></p>
                            <p class="card-text" id="Leadership_justifications"></p>
                        </div>
                    </div>
                    <div class="card mb-4 shadow-lg">
                        <div class="card-body">
                            <h4 class="card-title">Recommendation</h4>
                            <ul>
                                <li id="Recommendation1"></li>
                                <li id="Recommendation2"></li>
                                <li id="Recommendation3"></li>
                            </ul>
                        </div>
                    </div>
                    <div class="card mb-4 shadow-lg">
                        <div class="card-body">
                            <h4 class="card-title">Recommended Course</h4>
                            <p>Based on our analysis, we highly recommend the following course to enhance your skills:</p>
                            <div id="RecommendedCourse" class="mt-3">
                                <strong>Course Name:</strong> <span id="CourseRecommendation"></span><br>
                                <strong>Provider:</strong> <span id="CourseProvider"></span>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
    </div>

    <script>
document.getElementById('fileInput').addEventListener('change', function(event) {
    const file = event.target.files[0];
    if (file) {
        const formData = new FormData();
        formData.append('file', file);

        // Show loading indicator
        document.getElementById('loadingIndicator').style.display = 'block';

        fetch(window.location.href + "upload", {
            method: 'POST',
            body: formData,
        })
        .then(response => response.json())
        .then(data => {
            // Hide loading indicator
            document.getElementById('loadingIndicator').style.display = 'none';

            // Update your webpage with the response
            document.getElementById('Data_science_Rating').style.width = data.Data_science_Rating + "%";
            document.getElementById('Leadership_Rating').style.width = data.Leadership_Rating + "%" ;
            document.getElementById('Overall_Rating').style.width = data.Overall_Rating + "%";
            document.getElementById('Data_science_Rating_P').textContent = data.Data_science_Rating + "%";
            document.getElementById('Leadership_Rating_P').textContent = data.Leadership_Rating + "%";
            document.getElementById('Overall_Rating_P').textContent = data.Overall_Rating + "%";
            document.getElementById('Introduction').textContent = data.Introduction;
            document.getElementById('Data_science_justifications').textContent = data.Data_science_justifications;
            document.getElementById('Leadership_justifications').textContent = data.Leadership_justifications;
            document.getElementById('Recommendation1').textContent = data.Recommendation1;
            document.getElementById('Recommendation2').textContent = data.Recommendation2;
            document.getElementById('Recommendation3').textContent = data.Recommendation3;
            document.getElementById('CourseRecommendation').textContent = data.CourseRecommendation;
            document.getElementById('CourseProvider').textContent = data.CourseProvider;
        })
        .catch(error => {
            // Hide loading indicator
            document.getElementById('loadingIndicator').style.display = 'none';

            console.error('Error:', error);
        });
    }
});
    </script>




</body></html>
"""